In [6]:
#!source myenv/bin/activate
#!python3 --version
!which python3
from tabulate import tabulate
import pandas as pd
from sklearn.model_selection import train_test_split

/Users/adnan.alrawass/jupyter-1.0.0/myenv/bin/python3


In [ ]:
# getting dataset collected from ansible deffect dataset
file_path = './ansible.csv'
dfn = pd.read_csv(file_path)
dfn["label_before"]=1
dfn["label_after"]=0
#testing 
print(tabulate(dfn[9:14], headers='keys', tablefmt='grid'))


In [18]:
#lower case and removing space 
def clean_text(text):
    if pd.isna(text):
        return ''  # Return an empty string for NaN values
    elif isinstance(text, (int, float)):
        return str(text)  # Convert numeric values to strings
    else:
        return ' '.join(text.strip().lower().split())  # Remove multiple spaces




In [19]:
#removing space and lowercase implementation
dfn["Deleted Lines"] = dfn["Deleted Lines"].apply(clean_text)
dfn["Added Lines"] = dfn["Added Lines"].apply(clean_text)



In [20]:
#removing duplicates and null
dfn.dropna(subset=["Deleted Lines"], inplace=True)
dfn.dropna(subset=["Added Lines"], inplace=True)

dfn.drop_duplicates(subset=['Deleted Lines',"Added Lines"], inplace=True)

In [21]:

# Splitting into training and test, assuming train_id and test_id are defined
train, test = train_test_split(dfn, test_size=0.3, random_state=42)  # random_state ensures reproducibility

print(f"Total number of entries: {len(dfn)}")
print(f"Number of training entries: {len(train)}")
print(f"Number of test entries: {len(test)}")

Total number of entries: 3598
Number of training entries: 2518
Number of test entries: 1080


In [7]:
import json
from tqdm import tqdm

def create_data(set_data='train', directory='./'):
    data_list = []  # Use a list to store data dictionaries

    if set_data == 'train':
        data_df = train
        print(data_df)
    elif set_data == 'valid':
        # Use validation set size based on train/test split
        validation_size = int(len(train) * 0.2)  # Assumes 20% validation
        data_df = train.iloc[validation_size:]  # Select validation set from remaining entries
    elif set_data == 'test':
        data_df = test

    print(f"Number of entries in {set_data} set: {len(data_df)}")

    # Iterate over the DataFrame and append data for "Before" and "After" snippets
    for index, row in tqdm(data_df.iterrows()):
        code_before = str(row["Deleted Lines"])
        code_after = str(row["Added Lines"])
        label_before = int(row['label_before'])
        label_after = int(row['label_after'])
     

        # Check if code_before is not empty
        if code_before.strip():
            # Append a dictionary for "Before" code snippet
            data_list.append({
                'code': code_before,
                'label': label_before,
       
            })

        # Check if code_after is not empty
        if code_after.strip():
            # Append a dictionary for "After" code snippet
            data_list.append({
                'code': code_after,
                'label': label_after,
                
            })

    # Write data to a single JSONL file
    with open(f'{directory}/{set_data}.jsonl', 'w') as f:
        for item in data_list:
            json.dump(item, f)
            f.write('\n')  # Add newline after each JSON object

    return data_list

# Create JSONL files for training, validation, and test sets
train_data = create_data(set_data='train')
valid_data = create_data(set_data='valid')
test_data = create_data(set_data='test')

data = []

with open('./train.jsonl', 'r') as file:
    for line in file:
        # Parse JSON object from each line
        json_obj = json.loads(line)
        data.append(json_obj)

# Create DataFrame from the list of JSON objects
dfn = pd.DataFrame(data)
dfn.head(5)


NameError: name 'train' is not defined

In [1]:
%cd codebert-classification/code

/Users/adnan.alrawass/jupyter-1.0.0/project_snt/codebert-classification/code


/Users/adnan.alrawass/jupyter-1.0.0/myenv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
!python3 run.py \
    --output_dir=./lastansible \
    --tokenizer_name=microsoft/codebert-base \
    --model_name_or_path=microsoft/codebert-base \
    --do_train \
    --train_data_file='../../train.jsonl' \
    --eval_data_file='../../valid.jsonl' \
    --test_data_file='../../test.jsonl' \
    --num_train_epochs 150 \
    --block_size 512\
    --train_batch_size 128 \
    --eval_batch_size  128\
    --learning_rate 2e-5 \
    --max_grad_norm 1.0 \
    --seed 123456

/Users/adnan.alrawass/jupyter-1.0.0/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
05/20/2024 06:57:21 - WARNING - __main__ -   device: cpu, n_gpu: 0
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training/evaluation parameters %s Namespace(train_data_file='../../train.jsonl', output_dir='./lastansible', eval_data_file='../../valid.jsonl', test_data_file='../../test.jsonl', model_name_or_path='microsoft/codebert-base', tokenizer_name='microsoft/codebert-base', block_si

In [10]:
#testing 
!python run.py\
    --output_dir=./lastansible\
    --tokenizer_name=microsoft/codebert-base\
    --model_name_or_path=microsoft/codebert-base\
    --do_eval \
    --do_test \
    --train_data_file='../../train.json' \
    --eval_data_file='../../valid.jsonl' \
    --test_data_file='../../test.jsonl'  \
    --num_train_epochs 20 \
    --block_size 512 \
    --train_batch_size 128 \
    --eval_batch_size 128 \
    --learning_rate 2e-5 \
    --max_grad_norm 1.0 \
    --seed 123456  2>&1 | tee test.log

/Users/adnan.alrawass/jupyter-1.0.0/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
05/21/2024 08:37:29 - WARNING - __main__ -   device: cpu, n_gpu: 0
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training/evaluation parameters %s Namespace(train_data_file='../../train.json', output_dir='./lastansible', eval_data_file='../../valid.jsonl', test_data_file='../../test.jsonl', model_name_or_path='microsoft/codebert-base', tokenizer_name='microsoft/codebert-base', block_siz

In [12]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
def calculate_metrics(predictions_file, labels_file):
  """Calculates precision, recall, and accuracy from prediction and label files.

  Args:
      predictions_file: Path to the file containing predictions (one per line).
      labels_file: Path to the file containing labels (one per line).

  Returns:
      A dictionary containing precision, recall, and accuracy values.
  """
  with open(predictions_file, 'r') as f:
    predictions = [int(line.strip()) for line in f]
  with open(labels_file, 'r') as f:
    labels = [int(line.strip()) for line in f]

  precision = precision_score(labels, predictions)
  recall = recall_score(labels, predictions)
  accuracy = accuracy_score(labels, predictions)

  return {
      "precision": precision,
      "recall": recall,
      "accuracy": accuracy
  }


In [13]:
#with cleanning with testing 
predictions_file = "./lastansible/predictions.txt"
labels_file = "./lastansible//labels.txt"
metrics = calculate_metrics(predictions_file, labels_file)
print("Metrics:")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")

Metrics:
Precision: 0.9287
Recall: 0.8845
Accuracy: 0.9176


In [2]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, average_precision_score, matthews_corrcoef

def calculate_metrics(predictions_file, labels_file):
    """Calculates precision, recall, accuracy, AUC-PR, and MCC from prediction and label files.

    Args:
        predictions_file: Path to the file containing predictions (one per line).
        labels_file: Path to the file containing labels (one per line).

    Returns:
        A dictionary containing precision, recall, accuracy, AUC-PR, and MCC values.
    """
    with open(predictions_file, 'r') as f:
        predictions = [float(line.strip()) for line in f]
    with open(labels_file, 'r') as f:
        labels = [int(line.strip()) for line in f]

    precision = precision_score(labels, [round(p) for p in predictions])
    recall = recall_score(labels, [round(p) for p in predictions])
    accuracy = accuracy_score(labels, [round(p) for p in predictions])
    auc_pr = average_precision_score(labels, predictions)
    mcc = matthews_corrcoef(labels, [round(p) for p in predictions])

    return {
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy,
        "auc_pr": auc_pr,
        "mcc": mcc
    }

# with cleaning with testing
predictions_file = "./lastansible/predictions.txt"
labels_file = "./lastansible/labels.txt"
metrics = calculate_metrics(predictions_file, labels_file)
print("Metrics:")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"AUC-PR: {metrics['auc_pr']:.4f}")
print(f"MCC: {metrics['mcc']:.4f}")

Metrics:
Precision: 0.9287
Recall: 0.8845
Accuracy: 0.9176
AUC-PR: 0.8734
MCC: 0.8336
